# Формирования таблицы словаря 'Marks' для базы данных.

### 1. Подготовка

In [1]:

import time
import json
import datetime

import dateutil
import requests as rq
import pandas as pd
import numpy as np
import tqdm

from sqlalchemy import create_engine


### 2. Формирую переменные с нужными индексами марки и сотрудников, для формирования нужных тендеров.

Для этого подтягиваю из временных таблиц сформированных в других скриптах нужные данные.

In [ ]:

engine_pet = create_engine('mysql+mysqlconnector://'':''@''/tender_project')

df_marks_ = pd.read_sql(
    'SELECT * FROM tmp_dict_marks', 
    con = engine_pet, 
    index_col = '_id'
).index.to_list()

df_users_ = pd.read_sql(
    'SELECT * FROM tmp_dict_users', 
    con = engine_pet, 
    index_col = '_id'
).index.to_list()

df_marks = '&marks=' + '&marks='.join(df_marks_)
df_users = '&users=' + '&users='.join(df_users_)

date_first = (
    datetime.datetime.today() + 
    dateutil.relativedelta.relativedelta(
        years =- 1, 
        month = 1, 
        day = 1, 
        hour = 0, 
        minute = 0, 
        seconds = 0, 
        microseconds = 0
    )
)

unix_date_first = int(date_first.timestamp() * 1000)


### 3. Формирование переменных для подключения

In [ ]:

TOKEN = 'Bearer '
API = '/api/tenders/v2/getlist'
URL = 'https://tenderplan.ru' + API
headers = {
    'Authorization': TOKEN,
    'Accept': 'application/json'
}


### 4. Формирование функции get запроса для цилка и функции преобразования времени

In [4]:

if __name__ == '__main__':
    def f_lst_tender(page_):
        '''Функция принимает в качестве аргумента номер страницы и выводит данные get запроса'''
        response = rq.get(
            URL + 
            f'?page={str(page_)}' + 
            f'&fromPublicationDateTime={str(unix_date_first)}' + 
            df_marks + 
            df_users,
            headers = headers
        ).json()
        return response['tenders']

def f_date_fr_time(x):
    '''Принимает UNIX-время и возвращате дату в формате datetime'''
    return datetime.datetime.fromtimestamp(x // 1000)

def round_2(x):
    '''Функция округления числа'''
    return round(x, 2)


### 5. Загрузка данных через цикл WHILE (проверка на пустую страницу). 

На каждой итерации таймаут 0,08 секунд, так как в документации к API ограничение на кол-во запросов. Не более 250 запросов за 10 секунд или не более 800 запросов за 60 секунд

In [5]:

lst_df = []
start_time = time.time()
count_page = 0

while f_lst_tender(count_page) != []:
    end_time = time.time()
    df_lst_tenders = f_lst_tender(count_page)
    lst_df = lst_df + df_lst_tenders
    count_page += 1
    time.sleep(0.08)
    
end_time = time.time()
del df_lst_tenders

print(f'Кол-во секунд: {round(end_time - start_time, 0)}')
print(f'Кол-во запросов: {len(lst_df) / 50}')
print(f'Кол-во запросов в секунду: {len(lst_df) / 50 / (end_time - start_time)}')
print(f'Кол-во секунд на запрос: {(end_time - start_time) / (len(lst_df) / 50)}')


Кол-во секунд: 77.0
Кол-во запросов: 93.86
Кол-во запросов в секунду: 1.2139691800623906
Кол-во секунд на запрос: 0.8237441414687365


### 6. На данном этапе формируется несколько DataFrame:
1. df_participans - DataFrame участников тендера
2. df_marks_users - таблица фактов с пересечением марки, сотрудника и id тендера (несколько марок может быть на один тендер)
3. df_dict_tenders - справочник тенедров, были cJoinнены данные по стат показателям (мин, макс, ср, медиана по участникам тендера)

In [6]:

'''Создание всех DataFrame'''

'''Создание DataFrame участников'''
df_participants = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id', 
            'participants'
        ]
    )
    .set_index('_id')
    .explode('participants')
    .dropna(subset = 'participants')
)

df_participants = (
    pd.json_normalize(df_participants['participants'])
    .set_index(df_participants.index)
    .loc[
        :, 
        [
            'guid', 
            'name', 
            'price', 
            'region', 
            'score', 
            'winner'
        ]
    ]
)
    
df_participants = (
    df_participants
    .assign(
        **{
            'price': df_participants['price'].map(round_2),
            'score': df_participants['score'].map(round_2)
        }
    )
    .fillna(
        {
            'winner': np.nan,
            'guid': np.nan
        }
    )
)

'''Создание DataFrame стат. данных по участникам для справочника тендеров'''

agg_price_score = {
    'price': ['mean', 'std', 'min', 'max'],
    'score': ['mean', 'std', 'min', 'max']
}

df_stat_participants = (
    df_participants
    .groupby('_id')[['price', 'score']]
    .agg(agg_price_score)
    .round(2)
)

'''Перименование стобцов'''
df_stat_participants.columns = ['-'.join(x) for x in df_stat_participants.columns.values]

'''Создание DataFrame данных заказчика для справочника тендеров'''

df_customers = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id',
            'customers'
        ]
    )
    .set_index('_id')
    .explode('customers')
)

df_customers = (
    pd.json_normalize(df_customers['customers'])
    .set_index(df_customers.index)
    .astype(str)
    .groupby('_id')[['guid', 'name', 'region']]
    .agg(' '.join)
)

'''Создание DataFrame тендеров с денормализванными данными по меткам и сотрудникам'''

df_marks_users = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id',
            'marks',
            'users'
        ]
    )
    .set_index('_id')
    .explode('marks')
    .explode('users')
)

df_marks_users = df_marks_users.loc[
    (df_marks_users['marks'].isin(df_marks_)) & 
    (df_marks_users['users'].isin(df_users_))
]

'''Создание итого справочника по тендерам с данными заказчика и стат данными по участникам'''

df_dict_tenders = (
    pd.DataFrame(
        lst_df,
        columns = [
            '_id',
            'maxPrice',
            'status',
            'placingWay',
            'orderName',
            'publicationDate',
            'number',
            'region'
        ]
    )
    .set_index('_id')
    .join(
        df_customers, 
        rsuffix = '/customres'
    )
    .join(df_stat_participants)
)

df_dict_tenders = df_dict_tenders.assign(
    **{
        'publicationDate': df_dict_tenders['publicationDate'].map(f_date_fr_time)
    }
)


### 7. Загрузка данных во временные таблицы tmp

In [7]:

(
    df_participants
    .reset_index()
    .to_sql(
        'tmp_participants', 
        con = engine_pet, 
        if_exists = 'replace', 
        index = False
    )
)
time.sleep(1)
(
    df_marks_users
    .reset_index()
    .to_sql(
        'tmp_marks_users', 
        con = engine_pet, 
        if_exists = 'replace', 
        index = False
    )
)
time.sleep(1)
(
    df_dict_tenders
    .reset_index()
    .to_sql(
        'tmp_dict_tenders', 
        con = engine_pet, 
        if_exists = 'replace', 
        index = False
    )
)


-1

In [8]:

del df_stat_participants
del df_customers
del agg_price_score
del df_marks_
del df_users_


In [9]:

print(df_participants.head())
print(df_marks_users.head())
print(df_dict_tenders.head())


                                              guid  \
_id                                                  
69038e545d48d39bb4fb779a  591f1d8e0640fd1a867aa8df   
69038e545d48d39bb4fb779a  59527dce2567a52e73e4f9c3   
69038e545d48d39bb4fb779a  5952a8512567a52e73e5264a   
69038e545d48d39bb4fb779a  597f25682567a550cae09cc9   
69038e545d48d39bb4fb779a  5afe43132567a500016f8562   

                                                      name      price  region  \
_id                                                                             
69038e545d48d39bb4fb779a      ООО "ЦЕНТР ОПТОВОЙ ТОРГОВЛИ"  385986.21    66.0   
69038e545d48d39bb4fb779a                 ООО "РЕФБЕЛТРАНС"  315806.90    31.0   
69038e545d48d39bb4fb779a               ООО "КОТЛСПЕЦСТРОЙ"  280717.24    36.0   
69038e545d48d39bb4fb779a                        ООО "АНСА"  350896.55    27.0   
69038e545d48d39bb4fb779a  ИП МОРОЗОВ МАКСИМ. ВЛАДИМИРОВИЧ.  263172.41    61.0   

                          score winner  
_id         